<a href="https://colab.research.google.com/github/cmajorros/Voice-to-text-Using-OpenAI-Lib/blob/main/Youtube_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube
!pip install yt_dlp
!pip install SpeechRecognition
import os
import pytube
import yt_dlp
import speech_recognition as sr

In [ ]:
import os
import yt_dlp
import speech_recognition as sr
import pandas as pd

# Step 1: Extract video URLs from the YouTube channel using yt-dlp
def get_video_urls(channel_url):
    ydl_opts = {
        'extract_flat': True,
        'quiet': True,
        'force_generic_extractor': True,
    }
    video_urls = []
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(channel_url, download=False)
            entries = info.get('entries', [])
            video_urls = [entry['url'] for entry in entries if entry.get('url')]
    except Exception as e:
        print(f"Error fetching video URLs: {e}")
    return video_urls

# Step 2: Download the audio from each video
def download_audio(video_url, output_path="audio"):
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
                'preferredquality': '192',
            }],
            'outtmpl': os.path.join(output_path, '%(id)s.%(ext)s'),
            'quiet': True,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
    except Exception as e:
        print(f"Error downloading audio: {e}")

# Step 3: Convert audio to text using speech recognition
def audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Audio unintelligible"
    except sr.RequestError as e:
        return f"Error with the recognition service: {e}"

def process_channel_videos(channel_url):
    video_urls = get_video_urls(channel_url)
    os.makedirs('audio', exist_ok=True)
    os.makedirs('transcripts', exist_ok=True)

    data = []

    for video_url in video_urls:
        print(f"Processing {video_url}...")
        download_audio(video_url)

        video_id = video_url.split('=')[-1]
        audio_file = f"audio/{video_id}.wav"
        if os.path.exists(audio_file):
            transcript = audio_to_text(audio_file)
            with open(f"transcripts/{video_id}.txt", 'w') as f:
                f.write(transcript)

        # Add video ID and URL to data list
        data.append({'video_id': video_id, 'video_url': video_url})

    # Create DataFrame
    df = pd.DataFrame(data)
    print(df)
    return df

if __name__ == "__main__":
    channel_url = "https://www.youtube.com/@yourchannel"
    video_urls = get_video_urls(channel_url)
    print(video_urls)



['https://www.youtube.com/watch?v=khXNETeYyhU', 'https://www.youtube.com/watch?v=6hQs3Lm3hSE', 'https://www.youtube.com/watch?v=JdoobW_2jIk', 'https://www.youtube.com/watch?v=BXrJRiQoYcA', 'https://www.youtube.com/watch?v=_fr6PO6iMfo', 'https://www.youtube.com/watch?v=D-FUYDAlGYU', 'https://www.youtube.com/watch?v=e-0j5ozWNAE', 'https://www.youtube.com/watch?v=2_7dDBK3Jvo', 'https://www.youtube.com/watch?v=fLsgoIXuZcQ', 'https://www.youtube.com/watch?v=4iT5lyxilIg', 'https://www.youtube.com/watch?v=V0HwGnC1rU8', 'https://www.youtube.com/watch?v=mAIH3hUoxEE', 'https://www.youtube.com/watch?v=7qqJYxZqauQ', 'https://www.youtube.com/watch?v=QPCvVy0QBsA', 'https://www.youtube.com/watch?v=cafjAk7t5MM', 'https://www.youtube.com/watch?v=6FNaEUGooZE', 'https://www.youtube.com/watch?v=sr4S9SVcTHE', 'https://www.youtube.com/watch?v=5RJ_pZrV3VY', 'https://www.youtube.com/watch?v=grZbws1Kw_Q', 'https://www.youtube.com/watch?v=kGfUIOK87V8', 'https://www.youtube.com/watch?v=sM1QMz33HC4', 'https://www

In [ ]:
#!pip install pytube
from pytube import YouTube
# Function to get upload date
def get_upload_date(video_url):
    try:
        yt = YouTube(video_url)
        return yt.publish_date
    except Exception as e:
        return str(e)

# Get upload dates for all videos
data = []
for url in video_urls:
    date = get_upload_date(url)
    if date and date > pd.Timestamp('2023-01-01') and date < pd.Timestamp('2024-01-01') :
        data.append((url, date))

# Create DataFrame
df = pd.DataFrame(data, columns=['URL', 'Upload Date'])

# Show the DataFrame
print(df)

                                            URL Upload Date
0   https://www.youtube.com/watch?v=2_7dDBK3Jvo  2023-12-20
1   https://www.youtube.com/watch?v=fLsgoIXuZcQ  2023-12-20
2   https://www.youtube.com/watch?v=4iT5lyxilIg  2023-11-15
3   https://www.youtube.com/watch?v=V0HwGnC1rU8  2023-10-26
4   https://www.youtube.com/watch?v=mAIH3hUoxEE  2023-10-24
5   https://www.youtube.com/watch?v=7qqJYxZqauQ  2023-09-01
6   https://www.youtube.com/watch?v=QPCvVy0QBsA  2023-08-24
7   https://www.youtube.com/watch?v=cafjAk7t5MM  2023-08-11
8   https://www.youtube.com/watch?v=6FNaEUGooZE  2023-07-13
9   https://www.youtube.com/watch?v=sr4S9SVcTHE  2023-06-07
10  https://www.youtube.com/watch?v=5RJ_pZrV3VY  2023-05-24
11  https://www.youtube.com/watch?v=grZbws1Kw_Q  2023-05-12
12  https://www.youtube.com/watch?v=kGfUIOK87V8  2023-05-10
13  https://www.youtube.com/watch?v=sM1QMz33HC4  2023-04-21
14  https://www.youtube.com/watch?v=GrlRuOELJRw  2023-03-15
15  https://www.youtube.com/watch?v=DTG5

In [ ]:
from pytube import YouTube
from moviepy.editor import *
import os
# Directory to save downloads
download_dir = 'downloads'
os.makedirs(download_dir, exist_ok=True)

def download_and_extract_audio(video_url, output_path):
    try:
        # Download video
        yt = YouTube(video_url)
        video_stream = yt.streams.filter(file_extension='mp4').first()
        video_path = video_stream.download(output_path=output_path)

        # Extract audio from video
        video = VideoFileClip(video_path)
        audio_path = os.path.splitext(video_path)[0] + '.mp3'
        audio = video.audio
        audio.write_audiofile(audio_path)
        audio.close()
        video.close()

        print(f"Video downloaded and audio extracted to {audio_path}")
    except Exception as e:
        print(f"An error occurred for {video_url}: {e}")

# Loop through DataFrame and process each YouTube URL
for index, row in df.iterrows():
    video_url = row['URL']
    download_and_extract_audio(video_url, download_dir)

MoviePy - Writing audio in /content/downloads/Navigating the Landscape of a Fully Open Source Data Stack in 2023.mp3


MoviePy - Done.
Video downloaded and audio extracted to /content/downloads/Navigating the Landscape of a Fully Open Source Data Stack in 2023.mp3
MoviePy - Writing audio in /content/downloads/Open Source BI FTW   Building Compelling Dashboards with Apache Superset.mp3


MoviePy - Done.
Video downloaded and audio extracted to /content/downloads/Open Source BI FTW   Building Compelling Dashboards with Apache Superset.mp3
MoviePy - Writing audio in /content/downloads/Enhancing Custom Visualization Plugins in Apache Superset (Part 1 Overview).mp3


MoviePy - Done.
Video downloaded and audio extracted to /content/downloads/Enhancing Custom Visualization Plugins in Apache Superset (Part 1 Overview).mp3
MoviePy - Writing audio in /content/downloads/Preset Brief Demo August 2023.mp3


MoviePy - Done.
Video downloaded and audio extracted to /content/downloads/Preset Brief Demo August 2023.mp3
MoviePy - Writing audio in /content/downloads/Live Demo Superset 101.mp3


chunk:  31%|███       | 24474/78588 [00:29<01:36, 561.12it/s, now=None]

In [ ]:
!pip install openai-whisper


In [ ]:
import whisper
import os

# Function to convert audio file to text using Whisper
def audio_to_text(audio_path, model):
    result = model.transcribe(audio_path)
    return result['text']

# Load the Whisper model (use 'base' or other available models)
model = whisper.load_model('base')

# Directory containing the .wav files
directory_path = 'downloads/'

# Directory to save the .txt files
output_directory = 'content/speech_conversion/'

# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".mp3"):
        audio_path = os.path.join(directory_path, filename)
        text = audio_to_text(audio_path, model)

        # Save the text to a .txt file
        output_path = os.path.join(output_directory, f"{os.path.splitext(filename)[0]}.txt")
        with open(output_path, 'w') as file:
            file.write(text)

        print(f"Converted {filename} to text and saved to {output_path}")
